# Binary Logistic regression

 In our case, the probability $p_i$ of individual $i$ being classified as *yes* is, 
		       \begin{equation}
		           p_i = \mathbb{P}\Big( 1 \vert \beta, x_i \Big) = \dfrac{1}{1 + e^{-\beta^T x_i}} \quad \text{and} \quad 1 - p_i = \dfrac{e^{-\beta^T x_i}}{1 + e^{-\beta^T x_i}}.
		           \label{eq: Linear Logit probabilities}
		       \end{equation}

The objective function is expressed with the cross-entropy as follows            
            \begin{align}
		           f(\beta) & = \dfrac{1}{N} \sum_{n=1}^{N}  f_n(\beta) \\
                            & = - \dfrac{1}{N} \sum_{n=1}^{N} y_n\log\Big( \mathbb{P}(1 \vert \beta, x_n) \Big) + (1 - y_n)\log\Big( \mathbb{P}(0 \vert \beta , x_n) \Big) \\
                            & = - \dfrac{1}{N} \sum_{n=1}^{N} y_n\log\Big( p_n \Big) + (1 - y_n)\log\Big( 1 - p_n \Big)
		           \label{eq: Logit objective function}
		       \end{align}
		       where $y_{n} = 1$  if individual $n$ is classified *yes*, and $0$ otherwise, and $N$ is the dataset size.
               
Let's denote, the **individual loss**, 
$$ f_n(\beta) = - y_n\log\Big( p_n \Big) - (1 - y_n)\log\Big( 1 - p_n \Big)$$
So, the **gradient** is
$$ \nabla f_n(\beta) = (p_n - y_n)x_n \qquad \text{and} \qquad \nabla f(\beta) = \dfrac{1}{N} \sum_{n=1}^{N} (p_n - y_n) x_n$$
And, the **Hessian**, 
$$ \nabla^2 f(\beta) = \dfrac{1}{N} \sum_{n=1}^{N} p_n(1-p_n)x_n x_n^T $$

### In matrix form

Let's have,
 - $X = \Big( x_1 \vert \cdots \vert x_N  \Big) \in \mathbb{R}^{N \times p}$ is feature matrix, 
 - $ Y = \Big( y_1 \vert \cdots \vert y_N  \Big) \in \mathbb{R}^{N \times 1}$ is the target vector
 - $ P = \Big( p_1 \vert \cdots \vert p_N  \Big) \in \mathbb{R}^{N \times 1}$ is the probability vector
 
Therefore, 
\begin{align}
        P &= \dfrac{1}{1 + \exp.(-X\cdot \beta)} \\
    f(\beta) &= -\dfrac{1}{N} \Big( Y^T \ln(P) + (1 - Y)^Y\ln(1 - P) \Big) \\
    \nabla f(\beta) &= \dfrac{1}{N} \big( P - Y \big)^Y X \\
    \nabla^2 f(\beta) &= \dfrac{1}{N} X \text{Diag}\big( P \odot (1-P) \big) X^T \; (\text{it is positive semi-definite -- good})
\end{align}
where $\odot$ is the element-wise multiplication.

In [1]:
using Pkg

push!(LOAD_PATH, "C://Users//jerem//OneDrive//Desktop//GitHub//Code//FinalCode")

4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "C://Users//jerem//OneDrive//Desktop//GitHub//Code//FinalCode"

In [2]:
# for algebra
using LinearAlgebra, ForwardDiff

# For probability
using Random, Distributions

# For statistics
using OnlineStats, StatsBase, Statistics

# Data import 
using ZipFile, 
      CSV # use version 0.8.5

# for data processing
using DataFrames, CategoricalArrays
        
# Dysplay
using Plots
pyplot()

# For regression models
using Flux, GLM

# Import sklearn.metrics to julia
using PyCall
sklearn = pyimport("sklearn.metrics")

using Sofia, Geraldine

# Backend
plotly()

┌ Info: Precompiling Geraldine [c0fffcf4-b2fb-46e8-8a34-03ca56aefa66]
└ @ Base loading.jl:1278
┌ Warning: Package Geraldine does not have Distributions in its dependencies:
│ - If you have Geraldine checked out for development and have
│   added Distributions as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Geraldine
└ Loading Distributions into Geraldine from project dependency, future warnings for Geraldine are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fata

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

Plots.PlotlyBackend()

In [3]:
# read file from zip archive
bank_folder = ZipFile.Reader("bank.zip")

# identify the right file in zip
bank_full_csv = filter(x->x.name == "bank-full.csv", bank_folder.files)[1]

bank_full = CSV.File(bank_full_csv) |> DataFrame

close(bank_folder)

describe(bank_full)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,age,40.9362,18,39.0,95,0,Int64
2,job,,admin.,,unknown,0,String15
3,marital,,divorced,,single,0,String15
4,education,,primary,,unknown,0,String15
5,default,,no,,yes,0,String3
6,balance,1362.27,-8019,448.0,102127,0,Int64
7,housing,,no,,yes,0,String3
8,loan,,no,,yes,0,String3
9,contact,,cellular,,unknown,0,String15


In [4]:
# dataset with n-1 dummy variables

# For One-Hot-Encoding

# Remove duration
data_dummy = copy(bank_full[:, Not(:duration)])

# transform :y in 0/1
transform!(data_dummy, @. :y => ByRow(x -> x == "yes") => :y)

# Getting the column which of type String
cat_val = names(select(data_dummy, findall(col -> eltype(col) <: AbstractString, eachcol(data_dummy[:, Not(:y)]))))

left_aside = []

for val in cat_val
    temp = unique(data_dummy[:, val])
    ux = temp[1:length(temp)-1]
    append!(left_aside, temp[end])
    
    transform!(data_dummy, @. Symbol($val) => ByRow(isequal(ux)) .=> Symbol($val, "_", ux))
end
# data_one_hot_encoding = data_one_hot_encoding[:, Not(cat_val)]

rename!(data_dummy,Symbol("job_admin.") => :job_admin)
rename!(data_dummy,Symbol("job_blue-collar") => :job_blue_collar)
rename!(data_dummy,Symbol("job_self-employed") => :job_self_employed)

# deleting original columns to keep only One-Hot-Encoding
select!(data_dummy, Not(cat_val)) 

describe(data_dummy)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Integer,Float64,Integer,Int64,DataType
1,age,40.9362,18,39.0,95,0,Int64
2,balance,1362.27,-8019,448.0,102127,0,Int64
3,day,15.8064,1,16.0,31,0,Int64
4,campaign,2.76384,1,2.0,63,0,Int64
5,pdays,40.1978,-1,-1.0,871,0,Int64
6,previous,0.580323,0,0.0,275,0,Int64
7,y,0.116985,0,0.0,1,0,Bool
8,job_management,0.209197,0,0.0,1,0,Bool
9,job_technician,0.168034,0,0.0,1,0,Bool


In [5]:
# Splitting in training and testing sets (in DataFrame type)
sample = randsubseq(1:size(data_dummy,1), 0.8)
df_train_dummy = data_dummy[sample, :]

notsample = [i for i in 1:size(data_dummy,1) if isempty(searchsorted(sample, i))]
df_test_dummy = data_dummy[notsample, :];

# Creating feature matrix and target vectors (Matrix and vector forms)
N_train_dummy, n_features_dummy = size(df_train_dummy[:, Not(:y)])
N_test_dummy, _ = size(df_test_dummy[:, Not(:y)])

X_train_dummy = Matrix{Float64}(df_train_dummy[:, Not(:y)])
y_train_dummy = convert(Array, df_train_dummy[:, :y])
@assert N_train_dummy == size(X_train_dummy, 1)

X_test_dummy = Matrix{Float64}(df_test_dummy[: , Not(:y)])
y_test_dummy = convert(Array, df_test_dummy[:,:y]);
@assert N_test_dummy == size(X_test_dummy, 1)

In [6]:
@show size(X_train_dummy)
@show size(X_test_dummy);

size(X_train_dummy) = (36336, 41)
size(X_test_dummy) = (8875, 41)


In [7]:
dt = fit(ZScoreTransform, X_train_dummy, dims=1)

X_train_scaled = StatsBase.transform(dt, X_train_dummy)
X_test_scaled = StatsBase.transform(dt, X_test_dummy);

In [8]:
@show size(X_train_scaled)
@show size(X_test_scaled);

size(X_train_scaled) = (36336, 41)
size(X_test_scaled) = (8875, 41)


In [9]:
# Dataframe scaled
all_names = DataFrames.names(data_dummy[:, Not(:y)])

df_train_scaled = DataFrame(X_train_scaled, all_names)
df_train_scaled.y = y_train_dummy
df_test_scaled = DataFrame(X_test_scaled, all_names)
df_test_scaled.y = y_test_dummy

describe(df_test_scaled)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,age,-0.00224297,-2.06649,-0.182801,4.80898,0,Float64
2,balance,-0.00709788,-3.1235,-0.30633,33.5333,0,Float64
3,day,-0.0108581,-1.78012,0.0211142,1.82235,0,Float64
4,campaign,0.0084001,-0.572983,-0.247199,17.0194,0,Float64
5,pdays,0.0239753,-0.409618,-0.409618,8.06839,0,Float64
6,previous,0.0062455,-0.241222,-0.241222,12.2922,0,Float64
7,job_management,0.000819655,-0.514223,-0.514223,1.94463,0,Float64
8,job_technician,-0.00797723,-0.45035,-0.45035,2.22044,0,Float64
9,job_entrepreneur,-0.00306144,-0.184723,-0.184723,5.41336,0,Float64


## Creation Model

In [10]:
struct Model_Binary_LR{U <: IsUpdatable, T} <: Sofia.AbstractStochasticModel{U}
    X_train::Array{T,2} # matrix of sample training features (N_train x n_features)
    y_train::Array
    
    X_test::Array{T,2} # matrix of sample testing features (N_test x n_features)
    y_test::Array
    
    intercept::Bool # if we add intercept β_0
    
    n_features::Int # number of features in train set
    dim::Int # parameter vector  dimension
    
    N_train::Int # number samples for training
    N_test::Int # number samples for testing
    
    function Model_Binary_LR(X_train::Array{T,2}, y_train::Array, X_test::Array{T,2}, y_test::Array, intercept::Bool = true, upd::Bool = false) where T
        UPD = upd ? Updatable : NotUpdatable
        N_train, n_features = size(X_train)
        N_test, _ = size(X_test)
        
        if intercept
            dim = n_features + 1
        else
            dim = n_features
        end
        return new{UPD, T}(X_train, y_train, X_test, y_test, intercept, n_features, dim, N_train, N_test)
    end
end

In [11]:
mo_BinLR = Model_Binary_LR(X_train_scaled, y_train_dummy, X_test_scaled, y_test_dummy)

Model_Binary_LR{NotUpdatable,Float64}([0.2881213789254879 -0.44510832414184054 … -0.34744377399560616 -0.2037230961661914; -0.7479087289499474 -0.45409396578031636 … -0.34744377399560616 -0.2037230961661914; … ; 1.5125205973237297 -0.23244813869791292 … -0.34744377399560616 -0.2037230961661914; -0.3711705079043346 0.5339938129098575 … -0.34744377399560616 4.908488521440667], Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 1, 1, 1, 1, 1, 1, 0, 0], [1.6067051525851328 0.2584341359965992 … -0.34744377399560616 -0.2037230961661914; 0.5706750447096975 0.04643955363700311 … -0.34744377399560616 -0.2037230961661914; … ; -1.6897542815639797 -0.4171529945999158 … -0.34744377399560616 -0.2037230961661914; 0.9474132657553104 -0.18019829657788689 … -0.34744377399560616 -0.2037230961661914], Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 1, 0, 1, 1, 1, 1], true, 41, 42, 36336, 8875)

In [12]:
# Sigmoid function for probability calculations
function my_sigmoid(x)
   return 1/(1 + exp(-x)) 
end

# test
x = [1.0 ; 2.0 ; 3.0]
my_sigmoid.(x)

3-element Array{Float64,1}:
 0.7310585786300049
 0.8807970779778823
 0.9525741268224334

In [13]:
function proba_LR(β::AbstractArray , X::Matrix; intercept::Bool=true)
    """
    β: parameter vector
    X: feature matrix of dimension Nxp
    """
    if intercept
        N = size(X, 1)
        return my_sigmoid.(hcat(ones(N), X)*β)
    else
        return my_sigmoid.(X*β)
    end
end


function LR_loss(β::AbstractArray, X::Matrix, y::Vector; intercept::Bool = true, ϵ::Float64=1e-15)
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = clamp.(proba_LR(β, X; intercept = intercept), ϵ, 1-ϵ)
    
    loss = 0.0
    for i=1:N
       if y[i]
            loss += log(p[i])
        else
            loss += log(1 - p[i])
        end
    end
    
    return -(1/N)*loss
end

function LR_loss_inds(β::AbstractArray{T}, X::Matrix, y::Vector; intercept::Bool = true, ϵ::Float64=1e-15) where T
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = clamp.(proba_LR(β, X; intercept = intercept), ϵ, 1-ϵ)
    
    losses = zeros(T, N)
    for i=1:N
       if y[i]
            losses[i] += log(p[i])
        else
            losses[i] += log(1 - p[i])
        end
    end
    
    return -losses
end

function grad_LR(β::AbstractArray, X::Matrix, y::Vector; intercept::Bool = true, ϵ::Float64=1e-8) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    if intercept 
        # add row of 1 for bias term
        X_temp = hcat(ones(N), X) 
    else
        X_temp = X
    end 
    return (1/N)*transpose(X_temp)*( p .- y )
end

function grad_LR!(G, β::AbstractArray, X::Matrix, y::Vector; intercept::Bool = true) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    if intercept 
        # add row of 1 for bias term
        X_temp = hcat(ones(N), X) 
    else
        X_temp = X
    end 
    G[:] = (1/N)*transpose(X_temp)*( p .- y )
end

function grad_LR_inds(β::AbstractArray, X::Matrix, y::Vector; intercept::Bool = true, ϵ::Float64=1e-8) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    if intercept 
        # add row of 1 for bias term
        X_temp = hcat(ones(N), X) 
    else
        X_temp = X
    end 
    return transpose(X_temp)*Diagonal( p .- y )
end

function Hessian_LR(β::Array, X::Matrix, y::Vector; intercept::Bool = true) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    # Diagonal matrix with probas
    Diag = Diagonal(p.*(1 .- p))
    
    if intercept 
        # add row of 1 for bias term
        X = hcat(ones(N), X) 
    end 
    
    return (1/N)*transpose(X)*Diag*X
end


function Hessian_LR!(H, β::Array, X::Matrix, y::Vector; intercept::Bool = true) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    # Diagonal matrix with probas
    Diag = Diagonal(p.*(1 .- p))
    
    if intercept 
        # add row of 1 for bias term
        X = hcat(ones(N), X) 
    end 
    
    H[:,:] =  (1/N)*transpose(X)*Diag*X
end


function Hessian_LR_dot_v(β::Array, X::Matrix, y::Vector, v::Vector; intercept::Bool = true) 
    """
    β: parameter vector
    X: feature matrix (Nxp)
    y: target vector (N)
    intercept: β_0 or not
    ϵ: to avoid log(0)
    """
    N = size(X, 1) 
    
    @assert N == size(y, 1)
    
    p = proba_LR(β, X;  intercept = intercept)
    
    # Diagonal matrix with probas
    Diag = Diagonal(p.*(1 .- p))
    
    if intercept 
        # add row of 1 for bias term
        X = hcat(ones(N), X) 
    end 
    
    return (1/N)*transpose(X)*Diag*X*v
end

Hessian_LR_dot_v (generic function with 1 method)

In [14]:
function Sofia.Nobs(mo::Model_Binary_LR)
   return mo.N_train
end

Sofia.Nobs(mo_BinLR)

36336

In [15]:
beta = rand(mo_BinLR.dim) .* 0.01

N = 100

100

In [16]:
# definition F 
function Sofia.F(x::Array{T,1}, mo::Model_Binary_LR ; sample = 1:Nobs(mo)) where T
   return LR_loss(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept)
end

@show F(beta, mo_BinLR)

# definition 
function Sofia.Fs(x::Array{T,1}, mo::Model_Binary_LR ; sample = 1:Nobs(mo)) where T
   return LR_loss_inds(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept)
end

@show mean(Sofia.Fs(beta, mo_BinLR)) - F(beta, mo_BinLR)

Sofia.Fs(beta, mo_BinLR; sample = 1:2)

F(beta, mo_BinLR) = 0.6930293229671265
mean(Sofia.Fs(beta, mo_BinLR)) - F(beta, mo_BinLR) = -4.3298697960381105e-15


2-element Array{Float64,1}:
 0.6931308992541585
 0.6827176148099343

In [17]:
# ------------------- gradient -----------------------
function Sofia.grad!(x::Vector, mo::Model_Binary_LR, stack::Vector ; sample=1:Nobs(mo))
    stack[:] = grad_LR(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept)[:]
end

function Sofia.grad(x::Vector, mo::Model_Binary_LR ; sample=1:Nobs(mo))
    return grad_LR(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept) 
end

function Sofia.grads(x::Vector, mo::Model_Binary_LR ; sample=1:Nobs(mo))
    return grad_LR_inds(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept) 
end

function Sofia.grads!(x::Vector, mo::Model_Binary_LR, stack::Array{Vector, 1}; sample=1:Nobs(mo))
    result = grad_LR_inds(x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept)
    for i in 1:length(sample)
        stack[i] = result[:, i]
    end
end

In [18]:
stack = Array{Float64, 1}(undef, mo_BinLR.dim)
Sofia.grad!(beta, mo_BinLR, stack; sample=1:N)

# vector stack
# show(stdout, "text/plain", stack)

# diff with forward diff
@show norm(ForwardDiff.gradient(t -> Sofia.F(t, mo_BinLR ; sample=1:N), beta) - stack)

# diff with forward diff
norm(ForwardDiff.gradient(t -> Sofia.F(t, mo_BinLR ; sample=1:N), beta) - Sofia.grad(beta, mo_BinLR;sample=1:N))

norm(ForwardDiff.gradient((t->begin
                    #= In[18]:8 =#
                    Sofia.F(t, mo_BinLR; sample = 1:N)
                end), beta) - stack) = 6.654581044595713e-16


6.654581044595713e-16

In [19]:
# ----------------- Hessian ---------------------------

function Sofia.H!(x::Vector, mo::Model_Binary_LR, stack::Matrix; sample=1:Nobs(mo))
    Hessian_LR!(stack, x, mo.X_train[sample, :], mo.y_train[sample]; intercept = mo.intercept) 
end

function Sofia.Hdotv(x::Vector, mo::Model_Binary_LR, v::Vector; sample=1:Nobs(mo))
    return Hessian_LR_dot_v(x, mo.X_train[sample, :], mo.y_train[sample], v; intercept = mo.intercept) 
end

function Sofia.Hdotv!(x::Vector, mo::Model_Binary_LR, v::Vector, stack::Vector; sample=1:Nobs(mo))
    stack[:] = Hdotv(x, mo, v; sample = sample)
end

In [20]:
v = rand(mo_BinLR.dim)

Hdotv(beta, mo_BinLR, v; sample=1:N) - ForwardDiff.hessian(t -> Sofia.F(t, mo_BinLR; sample=1:N), beta)*v

42-element Array{Float64,1}:
 -1.3877787807814457e-16
  8.326672684688674e-17
 -2.7755575615628914e-17
 -3.608224830031759e-16
 -1.3877787807814457e-17
 -5.551115123125783e-17
 -1.3877787807814457e-17
 -2.7755575615628914e-17
  0.0
 -7.28583859910259e-17
  9.367506770274758e-17
 -1.0408340855860843e-17
 -4.996003610813204e-16
  ⋮
 -8.326672684688674e-17
  2.7755575615628914e-17
 -1.734723475976807e-17
 -2.0816681711721685e-17
 -2.0816681711721685e-17
 -6.938893903907228e-18
  4.85722573273506e-17
 -3.469446951953614e-17
  1.3877787807814457e-17
 -8.326672684688674e-17
 -1.3877787807814457e-16
  4.163336342344337e-17

In [21]:
################################################################################
#                   Tests minimization

x0 = rand(mo_BinLR.dim)*0.01

42-element Array{Float64,1}:
 0.008910219029502722
 0.002403365831957356
 0.00919205404152907
 0.0032599184632270783
 0.0046406822641160625
 0.005142473439559745
 0.005143991284235581
 0.004695636519118724
 0.0044190917873551784
 0.0008155462386316659
 0.0023370140838736165
 0.002721950112405256
 0.00592606933823002
 ⋮
 0.007822269736592833
 0.008608018421064637
 0.007245598285945845
 0.003953111792453361
 0.008356016497342016
 0.0022050691043421435
 0.002226221926406726
 0.0028927899700256355
 0.009847349584362787
 0.0003014650106725303
 0.005472179398716645
 0.008959946923102773

In [28]:
verbose = false

# ---------------- smoothing choice ----------------
# smoothing = Geraldine.NoSmoothing()
# smoothing = Geraldine.NaiveSmoothing()
smoothing = Geraldine.CumulativeDecreaseSmoothing{Float64}(maxIter=5)

# varStrategy = Geraldine.TrueVar{Float64}(smoothing)
varStrategy = Geraldine.FirstOrderVar{Float64}(smoothing)

samplingStrategy = Geraldine.DynamicSampling{Geraldine.IndComRN}(Sofia.Nobs(mo_BinLR), varStrategy)


# BTR
accBtr =  Accumulator(ValueAccumulator(), DeltaAccumulator(), FieldAccumulator(:iterCG), ParamAccumulator(), SamplingSize())

btr =  Geraldine.BTRStruct(;Nmax=1000, eps=1e-5, sam=typeof(samplingStrategy))

 state, accumulatorBtr = btr(mo_BinLR, copy(x0) , samplingStrategy, accumulator = accBtr, verbose = verbose)

 # Collecting results
 resultBTR = Geraldine.structToDict(accumulatorBtr)

samplingBTR = resultBTR[:SamplingSizeAccumulator]
fBTR = resultBTR[:ValueAccumulator]
paramBTR = resultBTR[:ParamAccumulator];

p1 = plot(samplingBTR[1])
p2 = plot(fBTR)

plot(p1, p2, layout = (2,1))

isOptimal


<!DOCTYPE html>
 
 
 Plots.jl